In [1]:
import pandas as pd 
from datetime import datetime, date

In [2]:
from aind_data_access_api.document_db import MetadataDbClient

API_GATEWAY_HOST = "api.allenneuraldynamics.org"
DATABASE = 'metadata_index'
COLLECTION = 'data_assets'

docdb_api_client = MetadataDbClient(
   host=API_GATEWAY_HOST,
   database=DATABASE,
   collection=COLLECTION,
)
print(docdb_api_client._base_url)

https://api.allenneuraldynamics.org/v1/metadata_index/data_assets


In [3]:
aggregate = [
  {
    "$match": {
      "session.session_type": "BCI single neuron stim",
      "data_description.data_level": "derived",
      "processing.processing_pipeline.data_processes.start_date_time":{"$gte":"2025-08-03"}
    }
  },
  {
    "$project": {
      "name": 1,
      "subject_id": "$data_description.subject_id",
      "genotype": "$subject.genotype", 
      "virus": "$procedures.subject_procedures.procedures.injection_materials.name",
      "date_of_birth": "$subject.date_of_birth", 
      "sex": "$subject.sex", 
      "session_type": "$session.session_type", 
      "session_time": "$session.session_start_time", 
      "stimulus_epochs": "$session.stimulus_epochs.stimulus_name", 
      "project_name": "$data_description.project_name", 
      "modality": "$data_description.modality.name",   
      "targeted_structure": "$session.data_streams.stack_parameters.targeted_structure", 
      "session_number": {
        "$filter": {
          "input": "$session.stimulus_epochs",
          "as": "epoch",
          "cond": { "$eq": ["$$epoch.stimulus_name", "single neuron BCI conditioning"] }
        }
      },
      "ophys_fov": {
            '$map': {
                'input': '$session.data_streams',
                'as': 'stream',
                'in': {
                    '$map': {
                            'input': '$$stream.ophys_fovs',
                            'as': 'fov',
                            'in': '$$fov.notes'
                    }
                }
            }
        }    
    }
  },
  {
    "$project": {
      "name": 1,
      "subject_id": 1,
      "genotype": 1,
      "virus": 1,
      "date_of_birth": 1, 
      "sex": 1, 
      "session_type": 1,
      "session_time": 1, 
      "stimulus_epochs": 1, 
      "project_name": 1, 
      "modality": 1,   
      "targeted_structure": 1, 
      "session_number": { "$arrayElemAt": ["$session_number.session_number", 0] },
      "ophys_fov": 1
    }
  },
  {'$unwind': {'path': '$ophys_fov', 'preserveNullAndEmptyArrays': False}},
  {'$unwind': {'path': '$ophys_fov', 'preserveNullAndEmptyArrays': False}},
  {'$unwind': {'path': '$virus', 'preserveNullAndEmptyArrays': False}}, 
  {'$unwind': {'path': '$virus', 'preserveNullAndEmptyArrays': False}},
  {'$unwind': {'path': '$virus', 'preserveNullAndEmptyArrays': False}},
  {'$unwind': {'path': '$modality', 'preserveNullAndEmptyArrays': False}},
  {'$unwind': {'path': '$targeted_structure', 'preserveNullAndEmptyArrays': False}}
  
]

records = docdb_api_client.aggregate_docdb_records(
    pipeline = aggregate,
)


In [4]:
df = pd.DataFrame(records)
df = df.drop_duplicates(subset="name")

df['session_date'] = df.apply(lambda x: datetime.fromisoformat(x['session_time']).date(), axis=1)
df['session_time'] = df.apply(lambda x: datetime.fromisoformat(x['session_time']).time(), axis=1)
df['date_of_birth'] = df.apply(lambda x: datetime.strptime(x['date_of_birth'], '%Y-%m-%d').date(), axis=1)
df['age'] = df.apply(lambda x: (x['session_date'] - x['date_of_birth']).days, axis=1)

order = ['project_name','session_type','_id','name','subject_id','genotype','virus','date_of_birth','sex','modality','session_date','age','session_time','targeted_structure','ophys_fov','session_number']
df = df[order]
df


,project_name,session_type,_id,name,subject_id,genotype,virus,date_of_birth,sex,modality,session_date,age,session_time,targeted_structure,ophys_fov,session_number
0,Brain Computer Interface,BCI single neuron stim,03f1b82e-4494-46bc-a959-50c4b14d13da,single-plane-ophys_731015_2025-01-28_17-40-57_...,731015,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,2024-03-14,Female,Planar optical physiology,2025-01-28,320,17:40:57.996000,Primary Motor Cortex,FOV_04,22.0
6,Brain Computer Interface,BCI single neuron stim,b28d6321-1b1b-4110-87fb-0e19325c1dfe,single-plane-ophys_767715_2025-02-17_17-41-50_...,767715,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,2024-09-15,Female,Planar optical physiology,2025-02-17,155,17:41:50.758000,Primary Motor Cortex,FOV_03,8.0
11,Brain Computer Interface,BCI single neuron stim,580448b2-4b93-4477-b6ac-6e2972792fe0,single-plane-ophys_772414_2025-01-27_15-57-05_...,772414,Camk2a-tTA/wt;tetO-GCaMP6s/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,2024-10-15,Female,Planar optical physiology,2025-01-27,104,14:23:26.693000,Primary Motor Cortex,FOV_01,5.0
16,Brain Computer Interface,BCI single neuron stim,c1be8449-7bef-4967-88f5-6a6ffc29e5a3,single-plane-ophys_772414_2025-02-06_16-47-44_...,772414,Camk2a-tTA/wt;tetO-GCaMP6s/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,2024-10-15,Female,Planar optical physiology,2025-02-06,114,16:47:44.152999,Primary Motor Cortex,FOV_02,13.0
21,Brain Computer Interface,BCI single neuron stim,15b395c0-1caf-4be4-a212-a13b9a4d1dfc,single-plane-ophys_772414_2025-02-10_11-15-26_...,772414,Camk2a-tTA/wt;tetO-GCaMP6s/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,2024-10-15,Female,Planar optical physiology,2025-02-10,118,11:15:26.609999,Primary Motor Cortex,FOV_03,14.0
26,Brain Computer Interface,BCI single neuron stim,b9a4c361-66b0-4cd5-9392-75d116ef3385,single-plane-ophys_731015_2025-01-31_20-37-19_...,731015,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,2024-03-14,Female,Planar optical physiology,2025-01-31,323,20:37:19.623000,Primary Motor Cortex,FOV_04,23.0
32,Brain Computer Interface,BCI single neuron stim,0162d41c-613c-4215-b0aa-9690f85a9fda,single-plane-ophys_731015_2025-01-28_18-56-35_...,731015,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,2024-03-14,Female,Planar optical physiology,2025-01-28,320,17:40:57.996000,Primary Motor Cortex,FOV_04,22.0
38,Brain Computer Interface,BCI single neuron stim,30006aee-36db-44d9-a1fb-1b6583619434,single-plane-ophys_740369_2025-01-09_17-18-37_...,740369,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,2024-05-03,Female,Planar optical physiology,2025-01-09,251,16:01:04.455000,Primary Motor Cortex,FOV_05,22.0
42,Brain Computer Interface,BCI single neuron stim,17ae1324-de6d-433e-9aec-7f174d22e89a,single-plane-ophys_740369_2025-01-13_17-31-04_...,740369,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,2024-05-03,Female,Planar optical physiology,2025-01-13,255,16:02:50.863999,Primary Motor Cortex,FOV_05,24.0
49,Brain Computer Interface,BCI single neuron stim,c32e7e98-1104-4e24-a830-53d54f06b518,single-plane-ophys_740369_2025-01-24_21-18-11_...,740369,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,2024-05-03,Female,Planar optical physiology,2025-01-24,266,20:08:49.286000,Primary Motor Cortex,FOV_05,25.0


In [5]:
df.to_csv('/data/metadata/bci_metadata.csv', index=False)

In [6]:
df.reset_index(inplace=True)

In [7]:
df['name'][3]

'single-plane-ophys_772414_2025-02-06_16-47-44_processed_2025-08-06_18-52-14'